# Reading rasters with rasterio


# Downscaling rasters thanks to dask

In this notebook we will look at a concrete case of data aggregation, with the example of changing the resolution using dask.

Dask is a Python library that enables calculations to be parallelized and large quantities of data to be handled in a scalable way, using available resources (CPU, memory, etc.). Unlike tools such as Pandas or NumPy, Dask allows you to work on data sets that exceed the available RAM memory by chunking the data into smaller pieces (chunks) and parallelizing calculations.
Dask works with lazy computation: instead of executing immediately, it builds a task graph. Calculations are only executed when the final result is explicitly requested (for example, by calling .compute()).

Dask offers a wide range of [modules](https://docs.dask.org/en/stable/#how-to-use-dask) (dask dataframe...) that can be used to distribute calculations. In this notebook we will focus on the use of [dask-arrays](https://docs.dask.org/en/stable/array.html) to manipulate raster data.

In order to change the resolution, we will look at two possible methods. One of them will prove less effective than the other, which will allow us to establish some general principles to follow for optimal use of Dask. The first one will use [map_blocks](https://docs.dask.org/en/stable/generated/dask.array.map_blocks.html) and the second one only [dask.array.mean](https://docs.dask.org/en/stable/generated/dask.array.mean.html)

For this tutorial, we will use a Sentinel-2 acquisition stored on the local disk. We will target the storage directory under the variable ``sentinel_2_dir``. Users can modify this directory and the associated paths.

## Python scripts

### Import libraries

First let's import libraries needed for this tutorial and create our dask [LocalCluster](https://docs.dask.org/en/stable/deploying-python.html#localcluster) which allow us to create workers and use [dask's dashboard](https://docs.dask.org/en/latest/dashboard.html).


In [2]:
from pathlib import Path

from typing import List, Tuple, Union, Dict
import dask.array as da
import numpy as np
import rasterio
import rioxarray as rxr
from dask import delayed
from dask.distributed import Client, LocalCluster
from rasterio.transform import Affine

from utils import create_map_with_rasters

cluster = LocalCluster()
client = Client(cluster)

print("Dask Dashboard: ", client.dashboard_link)
client

/usr/local/lib/python3.10/dist-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37319 instead
  warnings.warn(


Dask Dashboard:  http://127.0.0.1:37319/status


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:37319/status,
Dashboard: http://127.0.0.1:37319/status,Workers: 1
Total threads: 1,Total memory: 0.98 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41157,Workers: 1
Dashboard: http://127.0.0.1:37319/status,Total threads: 1
Started: Just now,Total memory: 0.98 TiB
Comm: tcp://127.0.0.1:44851,Total threads: 1
Dashboard: http://127.0.0.1:38405/status,Memory: 0.98 TiB
Nanny: tcp://127.0.0.1:46553,


Dask return an url where the dashboard is availaible (usually http://127.0.0.1:8787/status). This is not a tutorial on how to use this dashboard, but we recommend using it in a separate window while using this notebook.

### Open raster thanks to rioxarray

Here we are going to open the raster data required for this tutorial, the RGB bands from a Sentinel-2 acquisition. To do this, we're going to use rioxarray and, more specifically, the [open_rasterio](https://corteva.github.io/rioxarray/html/rioxarray.html#rioxarray-open-rasterio) method, which opens the images lazily (without loading data into memory) and returns a `dask.array` object. 
From this method we will use the ``chunks`` and ``lock`` arguments, which respectively set a chunk size and limit access to the data to one thread at a time to avoid read problems. Here ``chunks`` is set to ``True`` to allow dask to automatically size chunks.


## Calculation of the Average NDVI with dask

In this example, we will use what we have learned to: 

1. Read the data from the disk and stack them.
2. Calculate the associated NDVI, which combines multi-band information into a single band.
3. Reduce the information by calculating the average NDVI within a window.
4. Write the resulting image to the disk.

First, let's read the data we need to perform the NDVI.

In [3]:
def open_raster_and_get_metadata(raster_paths: List[str], chunks: Union[int, Tuple, Dict, None]):
    """
    Opens multiple raster files, extracts shared geospatial metadata, 
    and returns the concatenated data along with resolution and CRS info.

    Parameters:
    -----------
    raster_paths : List[str]
        Paths to the raster files.
    chunks : Union[int, Tuple, Dict, bool, None]
        Chunk sizes for Dask (bands, height, width).

    Returns:
    --------
    Tuple[dask.array.Array, float, float, float, float, Union[str, CRS]]
        Concatenated raster data, x and y resolution, top-left coordinates, and CRS.
    """
    results = []
    for raster_path in raster_paths:
        with rxr.open_rasterio(raster_path, chunks=chunks, lock=True) as tif:
            reprojection = tif
            transform = reprojection.rio.transform()
            crs = reprojection.rio.crs
            x_res = transform[0]
            y_res = -transform[4]
            top_left_x = transform[2]
            top_left_y = transform[5]
            results.append(reprojection)

    return da.concatenate(results), x_res, y_res, top_left_x, top_left_y, crs

# Paths to RGB Sentinel-2 bands
sentinel_2_dir = "/work/scratch/data/romaint/" # change it for your own directory

s2_1 = f"{sentinel_2_dir}/SENTINEL2A_20210415-105852-555_L2A_T31TCJ_C_V3-0_FRE_STACK.tif"
s2_2 = f"{sentinel_2_dir}/SENTINEL2B_20210407-104900-035_L2A_T31TCJ_C_V3-0/SENTINEL2B_20210407-104900-035_L2A_T31TCJ_C_V3-0_FRE_STACK.tif"
s2_b4 = f"{sentinel_2_dir}/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1_FRE_B4.tif"
s2_b8 = f"{sentinel_2_dir}/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1_FRE_B8.tif"
reading_chunks = True
#(-1,2200,2200)
input_data_array, x_res, y_res, top_left_x, top_left_y, crs = open_raster_and_get_metadata([s2_b4,s2_b8], reading_chunks)

When the data is read, we can express the NDVI calculation as if it were a numpy array. We add ``[None, :, :]`` to keep the shape as ``(bands, rows, cols)``. Then we can apply reduction on the dask.array and use ``compute()`` on it to triger the computation.

In [4]:
input_data_array

dask.array<concatenate, shape=(2, 10980, 10980), dtype=int16, chunksize=(1, 6111, 10980), chunktype=numpy.ndarray>

In [5]:
print(input_data_array.shape)
ndvi_array = (input_data_array[1] - input_data_array[0]) / (input_data_array[1] + input_data_array[0])[None, :, :]

(2, 10980, 10980)


In [6]:
%%time

mean_ndvi = ndvi_array.compute()

CPU times: user 165 ms, sys: 455 ms, total: 619 ms
Wall time: 2.06 s


In [7]:
def create_raster(data: np.ndarray, output_file: Path, x_res, y_res, top_left_x, top_left_y, crs):
    transform = Affine.translation(top_left_x, top_left_y) * Affine.scale(x_res, -y_res)
    with rasterio.open(
            output_file, "w",
            driver="GTiff",
            height=data.shape[1],
            width=data.shape[2],
            count=data.shape[0],
            dtype=data.dtype,
            crs=crs,
            transform=transform
    ) as dst:
        dst.write(data)

crs="EPSG:4326"
output_file = Path("ndvi_dask.tif")
quicklook_img = f"{sentinel_2_dir}/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1_QKL_ALL.jpg"
create_raster(ndvi_array, output_file, x_res , y_res,
                  top_left_x, top_left_y, crs)

# Calculate NDVI With OTB in python

In [16]:
import otbApplication as otb

sentinel_2_dir = "/work/scratch/data/romaint"
s2_b4 = f"{sentinel_2_dir}/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1_FRE_B4.tif"
s2_b8 = f"{sentinel_2_dir}/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1_FRE_B8.tif"
out_ndvi_otb_py="/work/scratch/data/romaint/img_ndvi_otb.tif"
#Compute NDVI with OTB in python
app_ndvi_otb = otb.Registry.CreateApplication("BandMath")
app_ndvi_otb.SetParameterStringList("il",[s2_b4,s2_b8])
app_ndvi_otb.SetParameterString("exp","(im2b1-im1b1)/(im2b1+im1b1)")
app_ndvi_otb.SetParameterString("out",out_ndvi_otb_py)
app_ndvi_otb.ExecuteAndWriteOutput()


Warning 1: Invalid value for NUM_THREADS: 


Writing /work/scratch/data/romaint/img_ndvi_otb.tif...: 100% [**************************************************] (7s)


0

# Calculate NDVI with OTB in C++
This part will call BandMath with the otb CLI to compare performances with the python swig interface

In [23]:
%%bash
otbcli_BandMath -il "/work/scratch/data/romaint/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1_FRE_B4.tif" "/work/scratch/data/romaint/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1_FRE_B8.tif" -exp "( im2b1 - im1b1 ) / ( im2b1 + im1b1 )" -out "/work/scratch/data/romaint/img_ndvi_otb_cpp.tif" 

Warning 1: Invalid value for NUM_THREADS: 


Writing /work/scratch/data/romaint/img_ndvi_otb_cpp.tif...: 100% [**************************************************] (5s)


# Compute SuperImpose with OTB

Superimpose does a resampling then a crop to have a new raster that has the same resolution as the reference input image. This app is using multithreading a lot, it is interesting to compare it with a solution like dask and full python / rasterio

In [ ]:
import otbApplication as otb

product_dir = "/work/scratch/data/romaint"
appSI = otb.Registry.CreateApplication("Superimpose")

appSI.SetParameterString("inr", "QB_Toulouse_Ortho_PAN.tif")
appSI.SetParameterString("inm", "QB_Toulouse_Ortho_XS.tif")
appSI.SetParameterString("out", "SuperimposedXS_to_PAN.tif")

appSI.ExecuteAndWriteOutput()